# Amazon EC2 and AMI with AWS CLI

This notebook shows examples of AWS CLI commands that perform common tasks for Amazon EC2. We will create Amazon EC2 instances from the Deep Learning AMI, install additional software, then save the new image and launch a new instance with the new package installed.

## Create and Display Amazon EC2 Key Pairs

You can use the AWS Command Line Interface (AWS CLI) to create, display, and delete your key pairs for Amazon EC2. You use key pairs to connect to an Amazon EC2 instance.

You must provide the key pair to Amazon EC2 when you create the instance, and then use that key pair to authenticate when you connect to the instance.

Let's start by defining your key name:

In [ ]:
key_name = <YOUR_KEY_HERE>

In [ ]:
!echo {key_name}

### Creating a Key Pair

To create a key pair, use the `create-key-pair` command with the `--query` option, and the `--output text` option to pipe your private key directly into a file.

In [ ]:
!aws ec2 create-key-pair --key-name {key_name} --query 'KeyMaterial' --output text > {key_name}.pem

Creates the key pair and stores the public key in AWS. Let's look at the local key:

In [ ]:
!cat {key_name}.pem

Your private key isn't stored in AWS and can be retrieved only when it's created. You can't recover it later. Instead, if you lose the private key, you must create a new key pair.

If you're connecting to your instance from a Linux computer, we recommend that you use the following command to set the permissions of your private key file so that only you can read it.

In [ ]:
!chmod 400 {key_name}.pem

### Displaying Your Key Pair

A "fingerprint" is generated from your key pair, and you can use it to verify that the private key that you have on your local machine matches the public key that's stored in AWS.

The fingerprint is an SHA1 hash taken from a DER-encoded copy of the private key. This value is captured when the key pair is created, and is stored in AWS with the public key. You can view the fingerprint in the Amazon EC2 console or by running the AWS CLI command aws ec2 `describe-key-pairs`.

In [ ]:
!aws ec2 describe-key-pairs --key-name {key_name}

## Create and Configure Security Groups for Amazon EC2

You can create a security group for your Amazon Elastic Compute Cloud (Amazon EC2) instances that essentially operates as a firewall, with rules that determine what network traffic can enter and leave. You can create security groups to use in a virtual private cloud (VPC), or in the EC2-Classic shared flat network.

### Creating a Security Group
You can create security groups associated with VPCs or for EC2-Classic.

#### EC2-VPC
The following example shows how to create a security group for a specified VPC.

First let's find the id of your default VPC.

In [ ]:
!aws ec2 describe-vpcs --query "Vpcs[].VpcId"

Set <tt>vpc_id</tt> to the variable <tt>VpcId</tt> listed above:

In [ ]:
vpc_id = <YOUR_VPC_ID_HERE>

Define SG name:

In [ ]:
sg_name = <YOUR_SECURITY_GROUP_NAME_HERE>

In [ ]:
!aws ec2 create-security-group --group-name {sg_name} --description "My security group" --vpc-id {vpc_id}

Define <tt>sg_id</tt> as returned above by <tt>GroupId</tt>.

In [ ]:
sg_id = <YOUR_SECURITY_GROUP_ID_HERE>

To view the initial information for a security group, run the `describe-security-groups` command.

In [ ]:
!aws ec2 describe-security-groups --group-ids {sg_id}

### Adding Rules to Your Security Group

When you run an Amazon EC2 instance, you must enable rules in the security group to enable incoming network traffic for your means of connecting to the image.

For example, if you're launching a Windows instance, you typically add a rule to allow inbound traffic on TCP port 3389 to support Remote Desktop Protocol (RDP). If you're launching a Linux instance, you typically add a rule to allow inbound traffic on TCP port 22 to support SSH connections.

Use the `authorize-security-group-ingress` command to add a rule to your security group. A required parameter of this command is the public IP address of your computer, or the network (in the form of an address range) that your computer is attached to, in CIDR notation.

**Note**

We provide the following service, https://checkip.amazonaws.com/, to enable you to determine your public IP address. To find other services that can help you identify your IP address, use your browser to search for "what is my IP address". If you connect through an ISP or from behind your firewall using a dynamic IP address (through a NAT gateway from a private network), your address can change periodically. In that case, you must find out the range of IP addresses used by client computers.

The following example shows how to add a rule for ssh (TCP port 22) to an EC2-VPC security group.

You can start by confirming your public address:

In [ ]:
!curl https://checkip.amazonaws.com

Save that as a variable `my_ip`:

In [ ]:
my_ip = <MY_IP>

With that information confirmed, you can add the range to your security group by running the `authorize-security-group-ingress` command.

In [ ]:
!aws ec2 authorize-security-group-ingress --group-id {sg_id} --protocol tcp --port 22 --cidr {my_ip}/32

To view the changes to the security group, run the `describe-security-groups` command.

In [ ]:
!aws ec2 describe-security-groups --group-ids {sg_id}

## Launch, List, and Terminate EC2 Instances

You can use the AWS Command Line Interface (AWS CLI) to launch, list, and terminate Amazon Elastic Compute Cloud (Amazon EC2) instances. You need a key pair and a security group. You also need to select an Amazon Machine Image (AMI) and make a note of the AMI ID.

### Launching an Instance
To launch an Amazon EC2 instance using the AMI you selected, use the `run-instances` command. You can launch the instance into a virtual private cloud (VPC), or if your account supports it, or into EC2-Classic.

Initially, your instance appears in the `pending` state, but changes to the `running` state after a few minutes.

We're going to use the Deep Learning AMI:

In [ ]:
ami_id = "ami-060865e8b5914b4c4"

We're going to launch the instance into our default vpc, and we need to pick a subnet.

In [ ]:
!aws ec2 describe-subnets

Choose one of the SubnetId's above and define below:

In [ ]:
subnet_id = <SUBNET_ID_HERE>

The following example shows how to launch a t2.medium instance in the specified subnet of a VPC.

In [ ]:
!aws ec2 run-instances --image-id {ami_id} --count 10 --instance-type t2.medium --key-name {key_name} --security-group-ids {sg_id} --subnet-id {subnet_id}

Note your instance id above:

In [ ]:
instance_id = <INSTANCE_ID_HERE>

### Listing Your Instances

You can use the AWS CLI to list your instances and view information about them. You can list all your instances, or filter the results based on the instances that you're interested in.

The following examples show how to use the describe-instances command.

The following command describes your instance and outputs the state of that instance.

In [ ]:
!aws ec2 describe-instances --instance-id {instance_id} --query "Reservations[].Instances[].State"

Keep checking your instance state. Once it is **running**, we're going to want to SSH into it. To do this, we need to check the public DNS name:

In [ ]:
!aws ec2 describe-instances --instance-id {instance_id} --query "Reservations[].Instances[].PublicDnsName"

Copy this name, and **STOP** executing cells and adhere to the following instructions:

1. Go back to the Jupyter Console
2. At the top right, click <tt>New</tt> and then click <tt>Terminal</tt>.
3. Enter `cd SageMaker/aws_ml_workshop/Lab1/`
4. SSH into your new instance via: `ssh -i <YOUR_KEY_HERE>.pem ubuntu@<YOUR_PUBLIC_DNS_HERE>`
5. Activate Pytorch env with `source activate pytorch_p36`
6. Download test RL script with `wget https://raw.githubusercontent.com/pytorch/examples/master/reinforcement_learning/reinforce.py`
7. Try running `python reinforce.py`
8. Install OpenAI Gym with `pip install gym`
9. Run `python reinforce.py` and validate the installation.

## Creating Amazon Machine Image (AMI)

Now that you've installed OpenAI gym into the Python 3.6 PyTorch Conda Environment, let's create an AMI so we can launch new instances with OpenAI gym installed in the Python 3.6 PyTorch Conda Environement by default.

First, define your AMI name:

In [ ]:
ami_name = <YOUR_GYM_AMI_NAME>

In [ ]:
!aws ec2 create-image --instance-id {instance_id} --name {ami_name}

Define `gym_ami_id` as the value returned above:

In [ ]:
gym_ami_id = <YOUR_GYM_AMI_ID>

Let's check the state of our AMI:

In [ ]:
!aws ec2 describe-images --image-ids {gym_ami_id} --query "Images[].State"

Once it's available, we can now launch a new EC2 instance from this AMI, with the same configurations as before:

In [ ]:
!aws ec2 run-instances --image-id {gym_ami_id} --count 1 --instance-type t2.medium --key-name {key_name} --security-group-ids {sg_id} --subnet-id {subnet_id}

Note the new instance id above:

In [ ]:
gym_instance_id = <YOUR_NEW_GYM_INSTANCE_ID>

Let's see the state of this instance:

In [ ]:
!aws ec2 describe-instances --instance-id {gym_instance_id} --query "Reservations[].Instances[].State"

Once this instance is running, let's grab the public DNS and ssh into like before to check that gym is already installed:

In [ ]:
!aws ec2 describe-instances --instance-id {gym_instance_id} --query "Reservations[].Instances[].PublicDnsName"

Copy this name, and **STOP** executing cells and adhere to the following instructions:

1. Go back to the Jupyter Console
2. At the top right, click <tt>New</tt> and then click <tt>Terminal</tt>.
3. Enter `cd SageMaker/aws_ml_workshop/Lab1/`
4. SSH into your new instance via: `ssh -i <YOUR_KEY_HERE>.pem ubuntu@<YOUR_NEW_PUBLIC_DNS_HERE>`
5. Activate Pytorch env with `source activate pytorch_p36`
6. Try running `python reinforce.py`

Verify that the installations you've made have persisted.

### Terminate Instances

Finally, let's terminate our demo instances.

In [ ]:
!aws ec2 terminate-instances --instance-ids {instance_id} {gym_instance_id}